# PART 2

## The structure of this notebook is as follows:

1. All the JSON files obtained from PART 1 are cnverted into a dataframe.
2. Country/Airport data set and Covid data set were used along with the flight data information to create interactive plots.

# Import all libraries required for interactive visualization

In [ ]:
import chart_studio.plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
%matplotlib inline
cf.go_offline()
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib.pyplot as plt
import seaborn as sns

# Converting the extracted JSON data into Dataframes

## Declaring Global Constants

In [ ]:
AIRPORT ='AMS'
AIRLINE = 'KLM'
ARRIVAL_FOLDER_LOCATION = 'C:\\Users\\Sreelatha\\Desktop\\ironHack\\Project 3\\flight_data\\2020\\arr'
DEPARTURE_FOLDER_LOCATION = 'C:\\Users\\Sreelatha\\Desktop\\ironHack\\Project 3\\flight_data\\2020\\dep'

## Import all the libraries required to get the information from the multiple json files

In [ ]:
import os 
import glob 
import requests
import json
import datetime 
import pandas as pd
from dateutil.relativedelta import relativedelta
from pandas.io.json import json_normalize


In [ ]:
#method to get all the files from ARRIVAL_FOLDER_LOCATION/current_month or DEPARTURE_FOLDER_LOCATION/current_month
def get_flight_json_files(path, current_month):

    path= path +'\\' + current_month  #build full path including current_month
    files = [f for f in glob.glob(path + "**/*.json", recursive=True)] #scan the folder and get  list of all json files under path
    return files

#method to exctract json file data into a DF 
def load_flight_data(file_list):
    new_flight_df = pd.DataFrame()
    #loop through files and extract flight JSON info into a dataframe
    for file in file_list:
        with open(file) as data_file:  #open next file for reading
                current_flight_df = pd.DataFrame()
                #read file into a dictionary
                flight_dict = json.loads(data_file.read())
                #get the pagination data only, this is because we need just the 'total' attribute data from pagination
                flight_dict = dict(flight_dict['pagination'])
                
                #move the file cursor to postion zero i.e to the start of the file so we can perform another read operation with json.load
                data_file.seek(0)
                #load JSON data from file.
                data = json.load(data_file)
                #normalize and flatten the flight data and put it in the DF
                current_flight_df = json_normalize(data,record_path=['data'])
                #add a new column to DF which takes the 'total' attribute value from pagination data and stores it in the flight DF
                current_flight_df['flight_total'] =  flight_dict['total']
                #print(current_flight_df)
                new_flight_df=  new_flight_df.append(current_flight_df)
               
                
    return new_flight_df

#main code start
# start date and end date
start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2020,3,27) 

#create pandas dataframe object to hold all the flight data from aviationstack
flight_pd = pd.DataFrame()
#create temp pandas dataframe object as a temp data holder during loop executions
temp_flight_pd = pd.DataFrame()


from datetime import date, timedelta

#dataframe creation from start date to end date in while loop, processing arrival data 
while start_date <= end_date: 
    current_month = start_date.strftime("%m")
    print(current_month)
    # get list of all JSON files containing arrival information for month current_month
    file_list = get_flight_json_files(ARRIVAL_FOLDER_LOCATION, current_month) 
    #DF containing the flight data for month X for arrivals
    temp_flight_pd = temp_flight_pd.append(load_flight_data(file_list))

    #move date forward by one month
    start_date += relativedelta(months=+1) 
    
#add a new column to qualify records as arrival records    
temp_flight_pd['flight.type']='Arr'
#add the arrival records to the primary flights DF
flight_pd = flight_pd.append(temp_flight_pd)

#dataframe creation from start date to end date in while loop, processing departure data
start_date = datetime.datetime(2020, 1, 1)
temp_flight_pd = pd.DataFrame()
   
while start_date <= end_date: 
    current_month = start_date.strftime("%m")
    print(current_month)
    # get list of all JSON files containing departure information for month current_month
    file_list = get_flight_json_files(DEPARTURE_FOLDER_LOCATION, current_month)
    #DF containing the flight data for month X for departures
    temp_flight_pd = temp_flight_pd.append(load_flight_data(file_list))
    #move date forward by one month
    start_date += relativedelta(months=+1)  

#add a new column to qualify records as dep records    
temp_flight_pd['flight.type']='Dep'
#add the dep records to the primary flights DF
flight_pd = flight_pd.append(temp_flight_pd)

#add a column to the flight DF which indicated month value for the flight record
flight_pd['flight_month'] = pd.DatetimeIndex(flight_pd['flight_date']).month

# Check if Dataframe is empty using empty attribute
if  flight_pd.empty == True:
     print('DataFrame is empty')
else:
    print('DataFrame is not empty')



In [ ]:
flight_pd.columns

### "flight_pd" has a lot of data with many unwanted data. So, I am only calling the columns I need in "flight_pd" and putting it in a dataframe named "flight_pd"

In [ ]:
filtered_flight_pd = flight_pd[['flight_date', 'flight_total','flight_month','airline.name','flight_status', 'departure.airport', 'arrival.airport','departure.iata', 'arrival.iata', 'flight.iata','flight.type']]
filtered_flight_pd.head

### Since the flight route dataframe, named flight_pd, does not have the name of the airports and the country they are located in; another dataframe was created based on another database. This dataframe named airports was downloaded from "https://openflights.org/data.html#airport" website.

In [ ]:
airports = pd.read_csv('../airport_data/airports.csv')
airports.head()

### Now I would like to concat both the dataframes as I would like to include the airport name and country name.

In [ ]:
# Merging the dataframes                       
nl_flights = pd.merge(filtered_flight_pd, airports, how ='inner', left_on ='arrival.iata',right_on='iata') 


# Sorting the date in European format:
nl_flights["flight_date"] = pd.to_datetime(nl_flights["flight_date"])

nl_flights


### Loading the COVID-19 Cases from Jan 2020 through March 2020 in the Netherlands. 
The data comes from " https://data.humdata.org/dataset/covid-19-global-travel-restrictions-and-airline-information. 
The data had all the countries in the world. I extracted only the cases from NL and put it in a dataframe.

In [ ]:
covid = pd.read_csv('../covid_data/covid.csv')
covid = covid[['Country/Region', 'Date', 'Value']]
covid = covid.loc[covid['Country/Region'] == "Netherlands"]


# Sorting the date in European format:
covid["Date"] = pd.to_datetime(covid["Date"])
covid = covid.sort_values(by="Date")


### Number of COVID Infections in NL in Jan, Feb, March 2020

In [ ]:
covid_one = covid.groupby(['Date']).sum().reset_index()
covid_one
# Interactive Plot:

import plotly.express as px
fig = px.line(covid_one, x="Date", y="Value", title="Number of infections in Jan, Feb, March")
fig.show()


In [ ]:
# Grouping by Date as there are multiple rows for the same date in NL.
covid = covid.groupby(['Date']).sum()

# Interactive Plot:
covid_cases = covid['Value']

covid_cases.iplot(kind='bar')

### Merging the flight and country database with COVID Database

In [ ]:
# Merging the dataframes
covid_flights = nl_flights[['flight_date','flight_total']]
covid_flights = covid_flights.drop_duplicates()
covid_flights = covid_flights.groupby('flight_date').agg('sum')

#covid_flights = pd.merge(covid_flights, covid, how ='inner', left_on ='flight_date',right_on='Date') 
#covid_flights = covid_flights.rename(columns={'Value': 'covid_infections', 'Date': 'covid_date', 'Country/Region': 'covid_country'})


covid_flights.head()



### Total number of flights for Jan, Feb, March 2020

In [ ]:
# Interactive Plot:
total_flights = covid_flights['flight_total']

total_flights.iplot(kind='bar')

### I would like to visualize the number of flights based on the flight status.

In [ ]:
# Interactive Plot:
status = nl_flights['flight_status'].value_counts()

status.iplot(kind='bar')


In [ ]:
cancelled_flight_filter = nl_flights['flight_status'] == 'cancelled'
new = nl_flights[cancelled_flight_filter].groupby('flight_date')['flight_status'].value_counts().reset_index(name = 'flights_cancelled').drop(['flight_status'], axis = 1) 



### Number of flights cancelled in NL in Jan, Feb, March 2020

In [ ]:

# Interactive Plot:

import plotly.express as px
fig = px.line(new, x="flight_date", y="flights_cancelled")
fig.show()



### Number of COVID Infections in NL vs FLight Cancellations in NL

In [ ]:

# covid data
x = covid_one.rename(columns={'Value': 'covid_infections'})

# cancelled flights data
new

# merge covid data and cancelled flights data

test = pd.merge(new, x, how ='inner', left_on ='flight_date',right_on='Date')
test = test.drop(['Date'], axis = 1)
# test.transpose()
test


import plotly.graph_objs as go

test = test.set_index('flight_date')
test.head()
trace0 = go.Scatter(x = test.index, y = test.flights_cancelled, mode = 'lines', name = "Flights Cancelled")
trace1 = go.Scatter(x = test.index, y = test.covid_infections, mode = 'lines', name = "Covid Infections")

data = [trace0, trace1]
layout = go.Layout(title = 'Covid Infections vs Flight Cancellations')

figure = go.Figure(data=data, layout=layout)
figure.show()

In [ ]:
# Pair plot:
sns.pairplot(test)
plt.show()

### Number of flight landed and cancelled in Jan 2020

In [ ]:
import plotly.express as px


month1 = nl_flights.loc[nl_flights['flight_month']== 1]
fig = px.histogram(month1, x="flight.type", color="flight_status")
fig.show()



### Number of flight landed and cancelled in March 2020

In [ ]:
import plotly.express as px
month3 = nl_flights.loc[nl_flights['flight_month']== 3]

fig = px.histogram(month3, x="flight.type", color="flight_status")
fig.show()

# Comparing the abobe and below graphs, there are a high number of flight cancelled in March 2020 compared to Jan 2020

In [ ]:
test.reset_index(inplace = True) 
test.head()

In [ ]:
@interact(Covid_Data = test['covid_infections'].unique(), 
          Flight_Date = test['flight_date'].unique())

def linechart(Covid_Data, Flight_Date):
    df = test[(test['covid_infections']==Covid_Data) & (test['flight_date']==Flight_Date)]
    grouped = test.groupby('flight_date').agg('sum')
    grouped.iplot(kind='bar', title='Covid Infections per Day')

# Insights and Conclusion:

From the study we can see that the number of flights cancelled and the number of COVID infections are realted. More number of infections and more number of flights cancelled were noticed in the month of March. 

The flights cancellations were not as significant as I was expecting. 


### Things to Improve:

1. There is a lot of potential to the data that was gatehred. A lot more interesting and interactive plotting can be performed to visualize the details.
2. The graphs should be normalized to get better visualization and a better perspective.





